In [1]:
!pip install gym

    100% |████████████████████████████████| 1.5MB 16.1MB/s 
    100% |████████████████████████████████| 1.0MB 18.7MB/s 
  Running setup.py bdist_wheel for gym ... - \ | / done
  Stored in directory: /root/.cache/pip/wheels/6c/3a/0e/b86dee98876bb56cdb482cc1f72201035e46d1baf69d10d028
Successfully built gym


In [0]:
import tensorflow as tf
import numpy as np
import gym

In [3]:
# Create the env
env_name = 'FrozenLake-v0'
env = gym.make(env_name)

/usr/local/lib/python3.6/dist-packages/gym/envs/registration.py:14: PkgResourcesDeprecationWarning: Parameters to load are deprecated.  Call .resolve and .require separately.
  result = entry_point.load(False)


In [9]:
# Get the action_space and observation_space
action_size = env.action_space.n
observation_size = env.observation_space.n

print ('Action Space Size: {}'.format(action_size),
        'Observation Space Size: {}'.format(observation_size))

Action Space Size: 4 Observation Space Size: 16


In [0]:
tf.reset_default_graph()

# The Q-Network

In [0]:
# Hyperparameters of the network
lr = 0.1
max_steps = 99
gamma = .95
total_train_episodes = 10000

# Exploration parameters
epsilon = .3
max_epsilon = 1.0
min_epsilon = 0.01
decay_rate = 0.01

# Other paramters
interval = 500
tboard_dir = './tboard/'

In [0]:
# Declare the placeholders
inputs = tf.placeholder(shape=[1, observation_size], dtype=tf.float32)
tqvals = tf.placeholder(shape=[1, action_size], dtype=tf.float32)

In [0]:
# Build the model

W1 = tf.get_variable('qweights', shape=[observation_size, action_size], 
                     initializer=tf.random_normal_initializer())

pout = tf.matmul(inputs, W1)
predict = tf.argmax(pout, axis=1)

In [0]:
# Define the loss function and optimizer
criterion = tf.reduce_sum(tf.square(tqvals - pout))
optimizer = tf.train.GradientDescentOptimizer(learning_rate=lr)
train_op = optimizer.minimize(criterion)

In [0]:
# TensorBoard Ops
def attach_summaries(var):
    mean = tf.reduce_mean(var)
    tf.summary.scalar('mean', mean)
    
    stddev = tf.sqrt(tf.reduce_mean(tf.square(var - mean)))
    tf.summary.scalar('stddev', stddev)
    
    tf.summary.scalar('max', tf.reduce_max(var))
    tf.summary.scalar('min', tf.reduce_min(var))
    tf.summary.histogram('histogram', var)
    
attach_summaries(W1)
attach_summaries(criterion)

In [105]:
## Traning the network

init = tf.global_variables_initializer()

reward_list = []
steps_list = []

with tf.Session(config=tf.ConfigProto(log_device_placement=True)) as sess:
    
    merged = tf.summary.merge_all()
    train_writer = tf.summary.FileWriter(tboard_dir + env_name + '/train/',
                                         sess.graph)
    
    sess.run(init)
    
    for episode in range(total_train_episodes):
        
        state = env.reset()
        done = False
        total_rewards = 0
        
        for step in range(max_steps):
            
            action, curr_qvals = sess.run([predict, pout], 
                                       feed_dict={inputs: np.identity(observation_size)[state:state+1]})
            action = action[0]
            
            exp_prob = np.random.uniform()
            if exp_prob < epsilon:
                action = env.action_space.sample()
                
                
            #print (action, curr_qvals)
            
            # Step with the chosen action
            new_state, rewards, done, info = env.step(action)
            
            # Get the qvalues for the new state
            next_qvals = sess.run([pout], 
                         feed_dict={inputs: np.identity(observation_size)[new_state:new_state+1]})
            
            # Get the target output for current state
            max_next_qvals = np.max(next_qvals)
            target_q = curr_qvals
            target_q[0, action] = rewards + (gamma * max_next_qvals)
            
            # Train the network using the target Q values
            _ = sess.run([train_op], feed_dict={inputs: np.identity(observation_size)[state:state+1],
                                                tqvals: target_q})
            
            
            total_rewards += rewards
            
            if done or step == max_steps:
                if episode % interval == 0:
                    env.render()
                    print ('Maximum steps: {}'.format(step))
                break
                
            state = new_state
        
        epsilon = min_epsilon + (max_epsilon - min_epsilon) * np.exp(-decay_rate * episode)
        
        reward_list.append(total_rewards)
        steps_list.append(step)
        
        if episode % interval == 0:
            print ('Episode {}/{}'.format(episode, total_train_episodes),
                   'Reward: {}'.format(total_rewards))
        
print ('Agent Trained successfully!!')
print ('Successful Episodes: {}/{}'.format(sum(reward_list), total_train_episodes))

  (Left)
SFFF
FHFH
FFFH
HFFG
Maximum steps: 2
Episode 0/10000 Reward: 0.0


/usr/local/lib/python3.6/dist-packages/numpy/core/_methods.py:26: RuntimeWarning: invalid value encountered in reduce
  return umr_maximum(a, axis, None, out, keepdims)


  (Left)
SFFF
FHFH
FFFH
HFFG
Maximum steps: 15
Episode 500/10000 Reward: 0.0
  (Left)
SFFF
FHFH
FFFH
HFFG
Maximum steps: 17
Episode 1000/10000 Reward: 0.0
  (Left)
SFFF
FHFH
FFFH
HFFG
Maximum steps: 4
Episode 1500/10000 Reward: 0.0
  (Left)
SFFF
FHFH
FFFH
HFFG
Maximum steps: 33
Episode 2000/10000 Reward: 0.0
  (Left)
SFFF
FHFH
FFFH
HFFG
Maximum steps: 25
Episode 2500/10000 Reward: 0.0
  (Left)
SFFF
FHFH
FFFH
HFFG
Maximum steps: 4
Episode 3000/10000 Reward: 0.0
  (Left)
SFFF
FHFH
FFFH
HFFG
Maximum steps: 11
Episode 3500/10000 Reward: 0.0
  (Left)
SFFF
FHFH
FFFH
HFFG
Maximum steps: 5
Episode 4000/10000 Reward: 0.0
  (Left)
SFFF
FHFH
FFFH
HFFG
Maximum steps: 5
Episode 4500/10000 Reward: 0.0
  (Left)
SFFF
FHFH
FFFH
HFFG
Maximum steps: 14
Episode 5000/10000 Reward: 0.0
  (Left)
SFFF
FHFH
FFFH
HFFG
Maximum steps: 11
Episode 5500/10000 Reward: 0.0
  (Left)
SFFF
FHFH
FFFH
HFFG
Maximum steps: 2
Episode 6000/10000 Reward: 0.0
  (Left)
SFFF
FHFH
FFFH
HFFG
Maximum steps: 3
Episode 6500/10000 Rewar

In [0]:
# Testing Hyperparameters
total_test_episodes = 100
max_steps = 99

interval = 1
show_each_episode = False

In [102]:
# Testing out the network

init = tf.global_variables_initializer()

test_reward_list = []

with tf.Session(config=tf.ConfigProto(log_device_placement=True)) as sess:
    
    merged = tf.summary.merge_all()
    test_writer = tf.summary.FileWriter(tboard_dir + env_name + '/test/',
                                        sess.graph)
    
    sess.run(init)
    
    for episode in range(total_test_episodes):
        
        state = env.reset()
        done = False
        final_reward = 0
        
        for step in range(max_steps):
            next_action = sess.run([predict], 
                          feed_dict={inputs: np.identity(observation_size)[state:state+1]})
            
            next_action = next_action[0][0]
            
            next_state, rewards, done, info = env.step(next_action)
            
            final_reward += rewards
            
            if done or step == max_steps:
                if episode % interval == 0 and show_each_episode:
                    env.render()
                    print ('Episode {}/{}'.format(episode+1, total_episodes),
                           'Rewards: {}'.format(all_rewards),
                           'Maximum steps: {}'.format(step+1))
                break
                
        test_reward_list.append(final_reward)
                
    print ('Successful Episodes: {}/{}'.format(sum(test_reward_list), total_test_episodes))

Successful Episodes: 3.0/100


In [40]:
# TensorBoard Setup
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip
get_ipython().system_raw('tensorboard --logdir=./tboard/FrozenLake-v0/ &')
get_ipython().system_raw('./ngrok http 6006 &')
!curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

--2018-11-30 08:37:55--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 52.45.111.123, 54.165.51.142, 54.174.228.92, ...
Connecting to bin.equinox.io (bin.equinox.io)|52.45.111.123|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5363700 (5.1M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip.1’

ngrok-stable-linux- 100%[===================>]   5.11M  9.77MB/s    in 0.5s    

2018-11-30 08:37:56 (9.77 MB/s) - ‘ngrok-stable-linux-amd64.zip.1’ saved [5363700/5363700]

Archive:  ngrok-stable-linux-amd64.zip
replace ngrok? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: ngrok                   
http://f794266a.ngrok.io
